# Training of Mixture Density Network

We trained a mixture density network that returns a complete probability distribution of the S-wave velocity in the upper 100m.  
Here we trained it for structures with 3 layers.
Use Love and Rayleigh dispersion curves together with their uncertainty vectors as input data.

In [ ]:
#import packages
import matplotlib.pyplot as plt
import math
import numpy as np
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from pickle import dump

import tensorflow as tf
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import optimizers
import keras
import tensorflow.keras.utils
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras import backend as k
from tensorflow_probability import distributions as tfd
from keras.utils.vis_utils import plot_model

### Set-up of neural network

In [ ]:
#function provided by Earp et al. (2020)
def log_sum_exp(x, axis=None):
    """Log-sum-exp trick implementation"""
    x_max = k.max(x, axis=axis, keepdims=True)
    return k.log(k.sum(k.exp(x - x_max), axis=axis, keepdims=True))+x_max

In [ ]:
#function provided by Earp et al. (2020)
### This is the key function that adjusts the network to produce an MDN
### It is a custom loss function
### Simple to translate into anything Python ML library you desire

def mean_log_Gaussian_like(y_true, parameters):
    """Mean Log Gaussian Likelihood distribution
    Note: The 'c' variable is obtained as global variable
    """
    components = k.reshape(parameters,[-1, c + 2, m])
    print("parameters are:",parameters)
    print("shape of parameters",parameters.shape)
    mu = components[:, :c, :]
    sigma = components[:, c, :]
    alpha = components[:, c + 1, :]

    alpha = k.clip(alpha,1e-8,1.)
    sigma = k.clip(sigma,1e-7,25.)
    exponent = k.log(alpha) - .5 * float(c) * k.log(2 * np.pi) \
    - float(c) * k.log(sigma) \
    - k.sum((k.expand_dims(y_true,2) - mu)**2, axis=1)/(2*(sigma)**2)

    log_gauss = log_sum_exp(exponent, axis=1)
    res = - k.mean(log_gauss)
    return res

In [ ]:
### X_train - training data corresponding to network inputs eg. dispersion curve data
### ytrain - training data corresponding to network outputs eg. 1D velocity model

x_size=100 # size of input data, dispersion curve sampled logarithmically with 100 points from 1-20 Hz
y_size = 3 # size of output data, 3 layers
c = y_size # this is used above in the function mean_log_Gaussian_like
m = 10 # number of mixtures in Gaussian, should be chosen according to the complexity of the problem

In [ ]:
### Network configuration ###

inp1 = Input((x_size,))  #input love dispersion curve
hidden_1 = Dense(271,activation=k.relu)(inp1)
drop_1 = Dropout(0.6)(hidden_1)

inp2=Input((x_size,))  #input love uncertainty vector
hidden_2=Dense(696,activation=k.relu)(inp2)
drop_2=Dropout(0.9)(hidden_2)

inp3 = Input((x_size,))  #input dispersion curve Rayleigh
hidden_3 = Dense(565,activation=k.relu)(inp3)
drop_3 = Dropout(0.1)(hidden_3)

inp4=Input((x_size,))  #input uncertainty vector Rayleigh
hidden_4=Dense(622,activation=k.relu)(inp4)
drop_4=Dropout(0.7)(hidden_4)

merged1=tf.keras.layers.concatenate([drop_1,drop_2]) #merge layers
hidden_5=Dense(254,activation=k.relu)(merged1)
drop_5 = Dropout(0.7)(hidden_5)

merged2=tf.keras.layers.concatenate([drop_3,drop_4])
hidden_6=Dense(479,activation=k.relu)(merged2)
drop_6 = Dropout(0.1)(hidden_6)

merged3=tf.keras.layers.concatenate([drop_5,drop_6])
hidden_7=Dense(814,activation=k.relu)(merged3)
drop_7 = Dropout(0.2)(hidden_7)

depth_lay=Dense(281,activation=k.relu)(drop_7)
drop_depth_lay = Dropout(0.2)(depth_lay)

#MDN layer
FC_mus   = Dense(y_size*m)(drop_7)  #mean
FC_sigma = Dense(m)(drop_7)  #stdev
FC_alpha = Dense(m,activation='softmax')(drop_7)  #weighting parameter

output  = tf.keras.layers.concatenate([FC_mus, FC_sigma, FC_alpha], axis=1) #output of MDN for probability of S-wave velocity
output2 = Dense(y_size,activation=k.relu)(drop_depth_lay) #output of layer depth

model = Model(inputs=[inp1,inp2,inp3,inp4], outputs=[output,output2]) 

### Set optimizer ###
adam = tf.keras.optimizers.Adam(learning_rate=0.001,decay=0.0, amsgrad=True)

mse = tf.keras.losses.MeanSquaredError()

### Compile model ###
model.compile(loss=[mean_log_Gaussian_like,mse],optimizer=adam,metrics=['accuracy']) 

In [ ]:
print(model.summary())

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#define callbacks
NAN = tensorflow.keras.callbacks.TerminateOnNaN() # If the training loss produces Nans, stop the training

### Load and preprocess training data

Dispersion curves and uncertainty vectors should be logarithmically resampled with 100 samples between 1 - 20 Hz corresponding to the frequency:
#### freq=np.logspace(0,1.3,100,base=10)

The phase velocity and uncertainty should be in km/s !! Layer depth is an intiger value.


In [ ]:
#load data from .npy array; insert your data here
input_disp_lov=np.load("love_disp.npy") #input Love Dispersion
input_disp_ray=np.load("ray_disp.npy") #input Rayleigh Dispersion
input_un_lov=np.load("love_un.npy") #input Love Uncertainty
input_un_ray=np.load("ray_un.npy") #input Rayleigh Uncertainty
output_svel=np.load("svel.npy") #output S-wave velocity
output_depth=np.load("depth.npy") #output depth of layer boundaries

print(input_disp_lov.shape) #shape should be: (number_training_models, 100)
print(input_disp_ray.shape)
print(input_un_lov.shape)
print(input_un_ray.shape)
print(output_svel.shape)
print(output_depth.shape)

In [ ]:
#print example dispersion curves with uncertainties

freq=np.logspace(0,1.3,100,base=10)
lov=input_disp_lov[0]
lov_un=input_un_lov[0]
ray=input_disp_ray[0]
ray_un=input_un_ray[0]


fig=plt.figure(figsize=(10,7))
ax1=fig.add_subplot(111)

ax1.errorbar(freq,lov,yerr=lov_un, label="Love")
ax1.errorbar(freq,ray,yerr=ray_un, label="Rayleigh")
plt.xlabel("Frequency (Hz)",fontsize=12)
plt.ylabel("Velocity (m/s)",fontsize=12)
plt.legend(fontsize=12)
plt.show()


In [ ]:
#preprocessing
#scale data 

scaler1 = StandardScaler() #for Love dispersion
scaler2 = StandardScaler() #for Love uncertainty
scaler3 = StandardScaler() #for Rayleigh dispersion
scaler4 = StandardScaler() #for Rayleigh uncertainty

input_disp_lov_scal = scaler1.fit_transform(input_disp_lov)
input_un_lov_scal = scaler2.fit_transform(input_un_lov)
input_disp_ray_scal = scaler3.fit_transform(input_disp_ray)
input_un_ray_scal = scaler4.fit_transform(input_un_ray)

#output doesn´t have to be scaled

In [ ]:
# save the scaler
dump(scaler1, open('./scaler1.pkl', 'wb'))
dump(scaler2, open('./scaler2.pkl', 'wb'))
dump(scaler3, open('./scaler3.pkl', 'wb'))
dump(scaler4, open('./scaler4.pkl', 'wb'))

In [ ]:
#split into training and test set

training_disp_lov=input_disp_lov_scal[:130000] #range depends on number of training data
testing_disp_lov=input_disp_lov_scal[130000:]

training_un_lov=input_un_lov_scal[:130000]
testing_un_lov=input_un_lov_scal[130000:]

training_disp_ray=input_disp_ray_scal[:130000]
testing_disp_ray=input_disp_ray_scal[130000:]

training_un_ray=input_un_ray_scal[:130000]
testing_un_ray=input_un_ray_scal[130000:]

training_svel=output_svel[:130000]
testing_svel=output_svel[130000:]

training_depth=output_depth[:130000]
testing_depth=output_depth[130000:]


print(np.shape(training_disp_lov))
print(np.shape(testing_disp_lov))
print(np.shape(training_un_lov))
print(np.shape(testing_un_lov))
print(np.shape(training_disp_ray))
print(np.shape(testing_disp_ray))
print(np.shape(training_un_ray))
print(np.shape(testing_un_ray))
print(np.shape(training_svel))
print(np.shape(testing_svel))

### Train model  
Number of epochs and batch size can be adapted

In [ ]:
#train model
results=model.fit([training_disp_lov,training_un_lov,training_disp_ray,training_un_ray], [training_svel,training_depth],batch_size=128, epochs=100,verbose=1, shuffle=True,validation_split=0.1,callbacks=[NAN])



In [ ]:
#save model
model.save('./model.h5')

In [ ]:
#evaluate model
loss_and_metrics = model.evaluate([testing_disp_lov,testing_un_lov,testing_disp_ray,testing_un_ray], [testing_svel,testing_depth])


In [ ]:
print(loss_and_metrics)

In [ ]:
# Loss Curves
fig=plt.figure(figsize=[8,6])

ax=fig.add_subplot(111)
ax.plot(results.history['loss'],'r',linewidth=3.0)
ax.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
plt.show()

### Evaluate performance  
Use previously unseen input data

In [ ]:
#test trained model on unseen data; insert your data here
unseen_disp_lov=np.load("love_disp_unseen.npy")
unseen_un_lov=np.load("love_un_unseen.npy")
unseen_disp_ray=np.load("ray_disp_unseen.npy")
unseen_un_ray=np.load("ray_un_unseen.npy")
unseen_vel=np.load("vel_unseen.npy")
unseen_depth=np.load("depth_unseen.npy")

print(np.shape(unseen_disp_lov))

In [ ]:
#scale data
unseen_disp_lov_scal=scaler1.transform(unseen_disp_lov)
unseen_un_lov_scal=scaler2.transform(unseen_un_lov)
unseen_disp_ray_scal=scaler3.transform(unseen_disp_ray)
unseen_un_ray_scal=scaler4.transform(unseen_un_ray)

In [ ]:
#predict velocity structure
predictions1,predictions2 = model.predict([unseen_disp_lov_scal,unseen_un_lov_scal,unseen_disp_ray_scal,unseen_un_ray_scal])

In [ ]:
print((predictions1.shape)) #30 values for means, 10 for stdevs and 10 for alphas (depends on number of kernels)
print((predictions2.shape)) #depth values

In [ ]:
#sort output for plotting of results

vel=np.linspace(0,2.5,6000) #samples for x-axis

errors=[]
predicted_means=[]
kernels=[]

for n in range(len(predictions1)):
    means=predictions1[n][0:y_size*m] #split prediction vector in means, stdev and alpha
    stdev=predictions1[n][y_size*m:y_size*m+m]
    alpha=predictions1[n][y_size*m+m:]
    model_mean=[]
    model_kernel= []
    for o in range(y_size):
        add=0
        for i in range(m):  #add all gaussian kernels
            kernel=alpha[i]*stats.norm.pdf(vel,means[o*m+i],abs(stdev[i]))
            add = add + kernel
        vel_estimate=vel[np.argmax(add)] #take maximum in gaussian kernel as mean velocity estimate for the layer
        model_mean.append(vel_estimate)
        model_kernel.append(add)
    predicted_means.append(model_mean)
    kernels.append(model_kernel)
    
    #compute error between predicted velocity value and actual value
    diff=[]
    for o in range(y_size):
        d=(model_mean[o]-unseen_vel[n][o]) 
        diff.append(d)
    errors.append(diff)
    

In [ ]:
#depth errors
depth_error=[]
for n in range(len(predictions2)):
    layer_err=[]
    for i in range(len(predictions2[n])):
        err=predictions2[n][i]-unseen_depth[n][i]
        layer_err.append(err)
    depth_error.append(layer_err)

print(np.shape(depth_error))

In [ ]:
def plot_kernels(prof):
    '''plot kernels for each layer
        
        prof = number of profile
    '''
    
    vel=np.linspace(0,2.5,6000)
    fig= plt.figure(figsize=(15,15))
    fig.subplots_adjust(wspace=0.3)

    num=1

    for i in range(y_size):
        ax=fig.add_subplot(4,y_size,num)
        ax.plot(vel,kernels[prof][i],color="black")
        x = ax.lines[0].get_xdata()
        y = ax.lines[0].get_ydata()
        plt.axvline(unseen_vel[prof][i],color='red',linestyle="--") #actual velocity within layer
        plt.xlabel("Vs (km/s)")
        plt.title("layer{}".format(num))
        num=num+1

In [ ]:
plot_kernels(990)

In [ ]:
def plot_profile(prof):
    '''plot profile with uncertainties
    
        prof = number of model
    '''
    
    x=np.linspace(0,2.5,6000) #samples for x-axis
    y=np.linspace(0,100,100) #samples for y-axis
    
    #transform layer depth into layer thickness
    #predicted layer depth
    layer_thickness_pred=[]
    lay1=int(predictions2[prof][0])
    layer_thickness_pred.append(lay1)

    for n in range(y_size-2):
        lay=int(predictions2[prof][n+1])-np.sum(layer_thickness_pred)
        layer_thickness_pred.append(lay)
    
    lay_last=100-np.sum(layer_thickness_pred)
    layer_thickness_pred.append(lay_last)
    print("predicted layer thickness:",layer_thickness_pred)
    

    #true layer depth
    layer_thickness_true=[]
    lay1_t=int(unseen_depth[prof][0])
    layer_thickness_true.append(lay1_t)

    for n in range(y_size-2):
        lay_t=int(unseen_depth[prof][n+1])-np.sum(layer_thickness_true)
        layer_thickness_true.append(lay_t)
    
    lay_last_t=100-np.sum(layer_thickness_true)
    layer_thickness_true.append(lay_last_t)
    print("true layer thickness:",layer_thickness_true)
    
    
    #mean velocity structure
    true_model=[]
    for n in range(y_size):
        for i in range(layer_thickness_true[n]):
            true_model.append(unseen_vel[prof][n])
        
    pred_model=[]
    for n in range(y_size):
        for i in range(layer_thickness_pred[n]):
            pred_model.append(predicted_means[prof][n])
            
    
    #probability distribution
    extent = [x.min(), x.max(), y.min(), y.max()]
    layers=np.flip(layer_thickness_pred) #layers have to be in reverse order!!

    z_list=[]
    for n in range(y_size):
        for i in range(layers[n]):
            pdf = kernels[prof][y_size-1-n]
            z_list.append(pdf)
            
    
    #plot figure
    fig=plt.figure(figsize=(4,7))

    ax1=fig.add_subplot(111)
    ax1.plot(true_model,y,label="True model")
    ax1.plot(pred_model,y,label="ML model")
    ax1.imshow(z_list, extent=extent, aspect = 'auto',cmap=plt.cm.Greys)
    plt.gca().invert_yaxis()
    plt.xticks(size=14)
    plt.yticks(size=14)
    plt.legend(fontsize=14)
    plt.xlim(0,2.5)
    plt.xlabel("Vs (km/s)",fontsize=14)
    plt.ylabel("Depth (m)",fontsize=14)

In [ ]:
plot_profile(990)

In [ ]:
#plot errors velocity
fig=plt.figure(figsize=(6,12))
fig.subplots_adjust(hspace=0.3)

num=1
for n in range(y_size):
    ax=fig.add_subplot(y_size,1,n+1)
    e=[]
    for i in range(len(errors)):
        e.append(errors[i][n])
    ax.hist(e, bins=100)
    plt.title("layer{}".format(num))
    plt.ylabel("number of models")
    plt.xlabel("velolcity difference (km/s)")
    num=num+1
    
plt.show()

In [ ]:
#plot errors depth
fig=plt.figure(figsize=(6,12))
fig.subplots_adjust(hspace=0.3)

num=1
for n in range(y_size):
    ax=fig.add_subplot(y_size,1,n+1)
    e=[]
    for i in range(len(depth_error)):
        e.append(depth_error[i][n])
    ax.hist(e, bins=100)
    plt.title("layer{}".format(num))
    plt.ylabel("number of models")
    plt.xlabel("depth difference (m)")
    num=num+1
    
plt.show()

In [ ]:
#Plot pearson correlation to visualize error for velocity estimate

line=np.linspace(0,2.5,500)

fig=plt.figure(figsize=(4,10))
fig.subplots_adjust(hspace=0.4)

num=1
for n in range(y_size):
    ax=fig.add_subplot(y_size,1,n+1)
    x=[]
    for i in range(len(predicted_means)):
        x.append(predicted_means[i][n])
    y=[]
    for i in range(len(unseen_vel)):
        y.append(unseen_vel[i][n])
        
    corr =stats.pearsonr(x, y)
    Z, xax,yax=np.histogram2d(x,y,bins=50,range=[[0,2.5],[0,2.5]])
    
    ax.pcolormesh(xax, yax, Z.T, cmap="Greys")
    plt.plot(line,line, color="red")
    plt.xlabel("Predicted velocity (km/s)",fontsize=12)
    plt.ylabel("True velocity (km/s)",fontsize=12)
    plt.title("layer {} - corr {}".format(num,"%0.3f" % (corr[0])))
    num=num+1
    
plt.show()

In [ ]:
#Plot pearson correlation to visualize error for depth estimate

line=np.linspace(0,100,100)

fig=plt.figure(figsize=(4,10))
fig.subplots_adjust(hspace=0.4)

num=1
for n in range(y_size):
    ax=fig.add_subplot(y_size,1,n+1)
    x=[]
    for i in range(len(predictions2)):
        x.append(predictions2[i][n])
    y=[]
    for i in range(len(unseen_depth)):
        y.append(unseen_depth[i][n])
        
    corr =stats.pearsonr(x, y)
    Z, xax,yax=np.histogram2d(x,y,bins=100,range=[[0,100],[0,100]])
    
    ax.pcolormesh(xax, yax, Z.T, cmap="Greys")
    plt.plot(line,line, color="red")
    plt.xlabel("Predicted thickness (m)",fontsize=12)
    plt.ylabel("True thickness (m)",fontsize=12)
    plt.title("layer {} - corr {}".format(num,"%0.3f" % (corr[0])))
    num=num+1
    
plt.show()

### References
Earp, S., Curtis, A., Zhang, X., & Hansteen, F. (2020). Probabilistic neural network tomography across Grane field (North Sea) from surface wave dispersion data. Geophysical Journal International, 223(3), 1741-1757.